In [1]:
# Optimization of an International Portfolio

In [2]:
! pip install 'pandas_datareader>=0.10'

In [3]:
# Import the necessary libraries 
from pandas_datareader.data import DataReader
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from typing import Dict, List
import yfinance as yf

In [4]:
# Create the formula to get the dates of the last 5 years.
def fnct_last_five_years():
    five_years = timedelta(days=365*5, )
    now = datetime.now()
    five_years_ago = now - five_years
    return now, five_years_ago

now, five_years_ago = fnct_last_five_years()
print(now)
print(five_years_ago)

2022-12-08 06:14:26.179087
2017-12-09 06:14:26.179087


In [5]:
def fnct_get_ticker_data(ticker: str, data_source: str = "yahoo") -> pd.DataFrame:
    """Get the data in the last 5 years for the specified ticker"""
    print(f'downloading data for {ticker}')
    now, five_years_ago = fnct_last_five_years()
    ticker_df = DataReader(ticker, data_source, five_years_ago, now)
    return ticker_df


def fnct_convert_currency(ticker_adj_close:pd.Series, currency:pd.Series) -> pd.Series:
    df = pd.DataFrame({"x":ticker_adj_close, "c":currency})
    # drop rows where all columns in subset are NA
    df = df.dropna(axis=0, how='all', subset=('x', 'c'))
    # fill with the value of the previous date's values
    df = df.fillna(method='ffill')
    # in case there are empty rows at the beginning, fill with following date's values
    df = df.fillna(method='bfill')
    return df['x'] * df['c']


def fnct_get_currency(*tickers):
    """obtain the currency for each ticker"""
    currencies = dict()
    for ticker in tickers:
        t = yf.Ticker(ticker, )
        currencies[ticker] = t.info["currency"]
    return currencies

def fnct_get_currencies_data(currencies:dict):
    """Use the output of get_currency to read the data for all the currencies once"""
    currencies_data = {}
    unique_currencies = set(currencies.values())
    ticker_map = {
        "USD": "MXN=X",
        "CAD": "CADMXN=X",
        "EUR": "EURMXN=X",
    }
    for c in unique_currencies:
        if c == "MXN":
            continue
        exchange = ticker_map[c]
        currencies_data[c] = fnct_get_ticker_data(exchange)['Close'] # {"USD":pd.Series, CAD:pd.Series}
    df = pd.DataFrame(currencies_data)
    df["MXN"] = 1
    return df

def fnct_get_tickers_in_mxn(currencies:List[str]):
    c_df = fnct_get_currencies_data(currencies)
    t_data = {}
    for ticker, coin in currencies.items():
        t = fnct_get_ticker_data(ticker)['Adj Close']
        t_in_mxn = fnct_convert_currency(t, c_df[coin])
        t_data[ticker] = t_in_mxn
    return pd.DataFrame(t_data)

In [6]:
def suma(a, b):
    c = a + b
    return(c)

suma(5, 7)


12

In [7]:
#tickers = ["BIMBOA.MX", "CEMEXCPO.MX", "PE&OLES.MX", "AMXL.MX", "HERDEZ.MX",
#    "BBD-B.TO", "RY.TO", "ENB.TO", 
#    "VWAGY", "SAP", "NSRGY", "EADSY",]
tickers = ['JCI', 'TGT', 'CMCSA', 'CPB', 'MO', 'APA', 
        'MMC', 'JPM', 'ZION', 'PSA', 'BAX', 'BMY', 'LUV', 
        'PCAR', 'TXT', 'TMO', 'DE', 'MSFT', 'HPQ', 'SEE', 
        'VZ', 'CNP', 'NI', 'T', 'BA']
currencies = fnct_get_currency(*tickers)


In [8]:
currencies

{'JCI': 'USD',
 'TGT': 'USD',
 'CMCSA': 'USD',
 'CPB': 'USD',
 'MO': 'USD',
 'APA': 'USD',
 'MMC': 'USD',
 'JPM': 'USD',
 'ZION': 'USD',
 'PSA': 'USD',
 'BAX': 'USD',
 'BMY': 'USD',
 'LUV': 'USD',
 'PCAR': 'USD',
 'TXT': 'USD',
 'TMO': 'USD',
 'DE': 'USD',
 'MSFT': 'USD',
 'HPQ': 'USD',
 'SEE': 'USD',
 'VZ': 'USD',
 'CNP': 'USD',
 'NI': 'USD',
 'T': 'USD',
 'BA': 'USD'}

In [9]:
data = fnct_get_tickers_in_mxn(currencies)


downloading data for MXN=X


downloading data for JCI


downloading data for TGT


downloading data for CMCSA


downloading data for CPB


...

downloading data for VZ


downloading data for CNP


downloading data for NI


downloading data for T


downloading data for BA


In [10]:
data

,JCI,TGT,CMCSA,CPB,MO,APA,MMC,JPM,ZION,PSA,...,TMO,DE,MSFT,HPQ,SEE,VZ,CNP,NI,T,BA
Date,,,,,,,,,,,,,,,,,,,,,
2017-12-11,632.916008,1044.919818,657.875617,786.951719,966.438887,694.657502,1470.073244,1740.889033,829.956590,3281.770630,...,3565.641038,2666.159243,1529.274390,349.521602,843.322438,786.150606,466.290013,440.380254,351.565851,5141.563033
2017-12-12,645.892594,1038.693296,680.310034,811.481578,973.280138,690.719112,1491.722946,1771.898449,843.710716,3333.899494,...,3587.377200,2677.804038,1544.915543,348.644532,847.397301,811.540299,461.489842,430.186848,365.211679,5296.767210
2017-12-13,630.723021,1058.045708,658.857889,795.618141,973.387255,686.443580,1457.661946,1735.351765,825.194566,3319.722448,...,3595.183932,2671.349787,1528.148524,343.647381,849.972209,800.356072,458.033877,427.311590,361.651064,5287.825266
2017-12-14,629.855845,1051.262113,671.619503,789.590134,971.373460,683.492023,1458.708689,1730.492660,820.564231,3313.133904,...,3557.559328,2641.144082,1524.366491,343.645619,850.579539,796.229992,464.027770,430.388382,360.700198,5353.002108
2017-12-15,631.370784,1062.044695,681.371203,805.891229,969.212287,683.979781,1459.480940,1753.991433,833.272949,3319.771779,...,3630.166926,2664.939465,1562.378563,346.436030,861.613074,800.806263,466.687540,430.637184,365.276384,5351.128231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-02,1303.019829,3140.101751,685.940839,1034.840985,911.080936,901.134211,3326.602673,2585.381059,999.453660,5658.150822,...,10769.232738,8523.761355,4878.099075,566.962792,1023.555286,730.318489,586.664954,527.367217,363.820269,3497.992075
2022-12-05,1302.730732,3003.006481,681.312124,1033.113346,914.683266,885.996450,3327.477838,2546.343286,938.524335,5610.406658,...,10800.396219,8468.234734,4849.624010,566.177117,1032.919549,718.527429,590.212057,533.226056,363.431060,3587.791546
2022-12-06,1303.584464,3057.803023,685.713063,1045.036982,911.325844,865.177767,3391.094551,2595.138863,929.666751,5699.286993,...,10857.222538,8642.903246,4834.109302,558.312804,1040.500994,727.522408,607.813521,549.043760,378.059226,3518.889134


In [11]:
pct_change_df = data.pct_change()

In [12]:
ann_mean_return = ((1 + pct_change_df.mean())**252)-1
print(ann_mean_return)

JCI      0.178790
TGT      0.273159
CMCSA    0.045560
CPB      0.101348
MO       0.023265
APA      0.300677
MMC      0.186900
JPM      0.120162
ZION     0.085053
PSA      0.139865
BAX      0.015102
BMY      0.108317
LUV     -0.012077
PCAR     0.147707
TXT      0.131001
TMO      0.262232
DE       0.293433
MSFT     0.274783
HPQ      0.152362
SEE      0.091843
VZ       0.009436
CNP      0.093007
NI       0.075405
T        0.044392
BA       0.039835
dtype: float64


In [13]:
ann_stdev = pct_change_df.std() * (252**(1/2))
print(ann_stdev)

JCI      0.276345
TGT      0.334745
CMCSA    0.276681
CPB      0.272912
MO       0.255273
APA      0.653936
MMC      0.232479
JPM      0.299277
ZION     0.354898
PSA      0.246295
BAX      0.257843
BMY      0.240622
LUV      0.365726
PCAR     0.262511
TXT      0.365208
TMO      0.272814
DE       0.322773
MSFT     0.295432
HPQ      0.355165
SEE      0.326904
VZ       0.208231
CNP      0.300705
NI       0.266577
T        0.243971
BA       0.462696
dtype: float64


In [14]:
pct_change_df.cov()

,JCI,TGT,CMCSA,CPB,MO,APA,MMC,JPM,ZION,PSA,...,TMO,DE,MSFT,HPQ,SEE,VZ,CNP,NI,T,BA
JCI,0.000303,0.000148,0.000153,0.000075,0.000117,0.000245,0.000154,0.000209,0.000214,0.000114,...,0.000132,0.000202,0.000159,0.000203,0.000183,0.000089,0.000156,0.000120,0.000130,0.000251
TGT,0.000148,0.000445,0.000140,0.000094,0.000114,0.000138,0.000136,0.000148,0.000145,0.000107,...,0.000153,0.000164,0.000181,0.000179,0.000130,0.000104,0.000114,0.000116,0.000113,0.000158
CMCSA,0.000153,0.000140,0.000304,0.000078,0.000111,0.000173,0.000132,0.000185,0.000177,0.000105,...,0.000118,0.000165,0.000178,0.000175,0.000144,0.000120,0.000149,0.000136,0.000152,0.000200
CPB,0.000075,0.000094,0.000078,0.000296,0.000105,-0.000015,0.000083,0.000064,0.000037,0.000106,...,0.000083,0.000051,0.000087,0.000048,0.000068,0.000103,0.000095,0.000109,0.000092,0.000014
MO,0.000117,0.000114,0.000111,0.000105,0.000259,0.000156,0.000101,0.000136,0.000133,0.000100,...,0.000082,0.000129,0.000097,0.000115,0.000116,0.000100,0.000119,0.000116,0.000118,0.000156
APA,0.000245,0.000138,0.000173,-0.000015,0.000156,0.001697,0.000162,0.000354,0.000407,0.000043,...,0.000110,0.000323,0.000183,0.000353,0.000232,0.000050,0.000237,0.000103,0.000166,0.000532
MMC,0.000154,0.000136,0.000132,0.000083,0.000101,0.000162,0.000214,0.000169,0.000141,0.000122,...,0.000150,0.000149,0.000174,0.000153,0.000142,0.000097,0.000157,0.000128,0.000111,0.000184
JPM,0.000209,0.000148,0.000185,0.000064,0.000136,0.000354,0.000169,0.000355,0.000330,0.000103,...,0.000125,0.000237,0.000182,0.000230,0.000203,0.000110,0.000194,0.000141,0.000164,0.000325
ZION,0.000214,0.000145,0.000177,0.000037,0.000133,0.000407,0.000141,0.000330,0.000500,0.000083,...,0.000094,0.000243,0.000143,0.000239,0.000201,0.000101,0.000176,0.000129,0.000161,0.000350
PSA,0.000114,0.000107,0.000105,0.000106,0.000100,0.000043,0.000122,0.000103,0.000083,0.000241,...,0.000112,0.000107,0.000127,0.000096,0.000115,0.000100,0.000146,0.000154,0.000097,0.000124


In [15]:

weight = np.full_like(ann_stdev,1/len(ann_stdev))
weight

array([0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
       0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
       0.04, 0.04, 0.04])

In [16]:
portaf_ann_stdev = ((weight @ pct_change_df.cov()) @ weight.T) ** (1/2)
portaf_ann_stdev

0.012757156744019147

In [17]:
pct_change

NameError: name 'pct_change' is not defined

In [ ]:
ticker = 'CADMXN=X'
data_source = 'yahoo'
goog = DataReader(ticker, data_source, five_years_ago, now)
goog

In [ ]:
ticker = 'CEMEXCPO.MX'
data_source = 'yahoo'
cemex = DataReader(ticker, data_source, five_years_ago, now)

In [ ]:
ticker = 'MXN=X'
data_source = 'yahoo'
mxn = DataReader(ticker, data_source, five_years_ago, now)

In [ ]:
df = pd.DataFrame({"goog":goog.Close, "cemex":cemex.Close, 'mxn':mxn['Close']})
# drop rows where all columns in subset are NA
df = df.dropna(axis=0, how='all', subset=('goog', 'cemex'))
# fill with the value of the previous date's values
df = df.fillna(method='ffill')
# in case there are empty rows at the beginning, fill with following date's values
df = df.fillna(method='bfill')

In [ ]:
#Add a column with google prices in MXN
df['goog_mxn'] = df.goog * df.mxn

In [ ]:
df.plot(y=['goog_mxn', "cemex"])

In [ ]:
df.corr()

In [ ]:
df.cov()

In [ ]:
df.std()

In [ ]:
df.std() * np.sqrt(252)

In [ ]:
df['goog_mxn_change'] = df.goog_mxn.pct_change()

In [ ]:
df.to_excel('example.xlsx')